In [ ]:
import os
import re
import unicodedata
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


from utils import *
from params import *
from Trainer import *

tf.__version__

In [ ]:
data = read_data("data/fra-eng", "fra.txt")
en_lines, fr_lines = zip(*data)

en_lines = [normalize(line) for line in en_lines]
fr_lines = [normalize(line) for line in fr_lines]

en_train, en_test, fr_train, fr_test = train_test_split(en_lines, fr_lines, shuffle=True, test_size=0.1)

print(" en_lines len {} \n fr_lines len {} \n en_train len {} \n en_test len {} \n fr_train len {} \n fr_test len {} " \
      .format(len(en_lines), len(fr_lines), len(en_train), len(en_test), len(fr_train), len(fr_test)))

fr_train_in = ['<start> ' + line for line in fr_train]
fr_train_out = [line + ' <end>' for line in fr_train]

In [ ]:
tokenizers, fr_data, en_data = tokenizeData(fr_train_in, fr_train_out, fr_test, en_train, en_test)

fr_tokenizer, en_tokenizer = tokenizers

en_vocab_size = len(en_tokenizer.word_index)+1
fr_vocab_size = len(fr_tokenizer.word_index)+1
print("en_vocab {}\nfr_vocab {}" .format(en_vocab_size, fr_vocab_size))

In [ ]:
# training without distribution
#train(en_data, fr_data, fr_test, (en_vocab_size, fr_vocab_size), fr_tokenizer)

In [ ]:
#training with distribution
distributedTrain(en_data, fr_data, fr_test, (en_vocab_size, fr_vocab_size), fr_tokenizer)